In [217]:
import cv2
import numpy as np
from tkinter import filedialog
from tkinter import *
#trackbar callback fucntion to update HSV value
def callback(x):
    global blue_low,blue_high,green_low,green_high,red_low,red_high,area_l,area_h
    #assign trackbar position value to H,S,V High and low variable
    blue_low = cv2.getTrackbarPos('low blue','controls')
    blue_high = cv2.getTrackbarPos('high blue','controls')
    green_low = cv2.getTrackbarPos('low green','controls')
    green_high = cv2.getTrackbarPos('high green','controls')
    red_low = cv2.getTrackbarPos('low red','controls')
    red_high = cv2.getTrackbarPos('high red','controls')
    area_l = cv2.getTrackbarPos('area_l','controls')
    area_h = cv2.getTrackbarPos('area_h','controls')


#create a seperate window named 'controls' for trackbar
cv2.namedWindow('controls',2)
cv2.resizeWindow("controls", 550,10);


#global variable
blue_low = 12
blue_high = 83
green_low= 30
green_high = 232
red_low= 0
red_high = 160
area_l=100
area_h=1000

#create trackbars for high,low H,S,V 
cv2.createTrackbar('low blue','controls',0,255,callback)
cv2.createTrackbar('high blue','controls',179,255,callback)

cv2.createTrackbar('low green','controls',0,255,callback)
cv2.createTrackbar('high green','controls',255,255,callback)

cv2.createTrackbar('low red','controls',0,255,callback)
cv2.createTrackbar('high red','controls',255,255,callback)
cv2.createTrackbar('area_l','controls',0,5000,callback)
cv2.createTrackbar('area_h','controls',5000,5000,callback)


root=Tk()
# Leer la imagen
root.filename1 =  filedialog.askopenfilename(initialdir = r"Users\ASUS\Desktop\zerodayssoftware\deteccion de terrenos",title = "Select file",filetypes = (("image",["*.png","*.jpg","*.jpeg","*.tiff","*.txt"]),("all files","*.*")))
filenames=root.filename1
root.destroy()
while(1):
    #read source image
    img=cv2.imread(filenames)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    bgr_low=np.array([blue_low, green_low, red_low], np.uint8)#np.array([76, 0, 136], np.uint8)
    bgr_high=np.array([blue_high, green_high, red_high], np.uint8)#np.array([255, 255, 255], np.uint8)
    mask=cv2.inRange(hsv,bgr_low,bgr_high)
    # mask = cv2.erode(mask, None, iterations=0)
    # mask = cv2.dilate(mask, None, iterations=8)
    img = cv2.bitwise_and(img, img, mask=mask)
    H,W,d=img.shape
    #convert sourece image to HSC color mode
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     hsv_low = np.array([H_low, S_low, V_low], np.uint8)
#     hsv_high = np.array([H_high, S_high, V_high], np.uint8)
#     #making mask for hsv range
#     mask = cv2.inRange(hsv, hsv_low, hsv_high)
#     print (mask)
    #masking HSV value selected color becomes black
    res = cv2.bitwise_and(img, img, mask=mask)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cont in contours:
        areas=cv2.contourArea(cont)
        if areas>area_l and areas<area_h:
            (x,y,w,h)=cv2.boundingRect(cont)
    #         centroides.append([1-(x+.5*w)/W,(y+.5*h)/H])
    #         coordenadas.append([1-(x)/W,(y)/H,1-(x+w)/W,(y+h)/H])
            cv2.rectangle(res, (x,y), (x+w,y+h), (0,255,0),1,cv2.LINE_AA)
            cv2.circle(res,(int(x+w/2),int(y+h/2)),10,(0,255,0))
    #show image
    R=300
    mask1=cv2.resize(mask, (R, int(R*(H/W))))
    res1=cv2.resize(res, (R, int(R*H/W)))
    cv2.imshow('mask',mask1)
    cv2.imshow('res',res1)
    #waitfor the user to press escape and break the while loop 
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
print([blue_low, green_low, red_low]+[blue_high, green_high, red_high])
#destroys all window
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:862: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:862: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:862: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


[0, 0, 0, 179, 255, 255]


## Codigo para generar shp de poligonos apartir de contornos

In [ ]:
import geopandas as gpd
import cv2
import numpy as np
from tkinter import filedialog
from tkinter import *
from shapely.geometry import Polygon
import pandas as pd
import tqdm
root=Tk()
root.filename1 =  filedialog.askopenfilename(initialdir = r"Users\ASUS\Desktop\zerodayssoftware\deteccion de terrenos",title = "Select file",filetypes = (("image",["*.png","*.jpg","*.jpeg","*.tiff","*.txt"]),("all files","*.*")),multiple=True)
filenames=root.filename1
toluca=gpd.read_file("C:/Users/ASUS/Documents/GobiernoEdomex/Agua/geoshapes/Full manzanas/Manzana_Toluca.shp")
root.destroy()
geometry=[]
for file in tqdm.tqdm(filenames):
    img = cv2.imread(file)
    cve=file.split("/")[-1].split(".")[0]
    array=[]
    for line in str(toluca[toluca["cve_cat"]==cve]["geometry"].values[0]).replace("POLYGON Z ","").replace("(","").replace(")","").split(","):
        array.append(line.split())
    shape=pd.DataFrame(array,columns=["Y","X","Z"])
    shape["X"]=shape["X"].astype("float64")
    shape["Y"]=shape["Y"].astype("float64")
    minx=float(shape["X"].min())
    maxx=float(shape["X"].max())
    miny=float(shape["Y"].min())
    maxy=float(shape["Y"].max())
    H,W,D=img.shape
    H_low = 10
    H_high = 109
    S_low= 63
    S_high = 255
    V_low= 73
    V_high = 255
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_low = np.array([H_low, S_low, V_low], np.uint8)
    hsv_high = np.array([H_high, S_high, V_high], np.uint8)
    mask = cv2.inRange(hsv, hsv_low, hsv_high)
    mask=cv2.blur(mask,(5,5))
    res = cv2.bitwise_and(img, img, mask=mask)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    centroides=[]
    coordenadas=[]
    for cont in contours:
        area=cv2.contourArea(cont)
        if area>500 and area<5000:
            (x,y,w,h)=cv2.boundingRect(cont)
            centroides.append([1-(x+.5*w)/W,(y+.5*h)/H])
            coordenadas.append([1-(x)/W,(y)/H,1-(x+w)/W,(y+h)/H])
            cv2.rectangle(res, (x,y), (x+w,y+h), (0,255,0),1,cv2.LINE_AA)
            cv2.circle(res,(int(x+w/2),int(y+h/2)),10,(0,255,0))
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    cv2.waitKey()
    cv2.destroyAllWindows()
    rectangulo=pd.DataFrame(coordenadas,columns=["y1","x1","y2","x2"])
    rectangulo["x1"]=maxx-rectangulo["x1"]*(maxx-minx)
    rectangulo["y1"]=maxy-rectangulo["y1"]*(maxy-miny)
    rectangulo["x2"]=maxx-rectangulo["x2"]*(maxx-minx)
    rectangulo["y2"]=maxy-rectangulo["y2"]*(maxy-miny)
    for i in range(len(rectangulo)):
        geometry.append(Polygon([(rectangulo["y1"][i],rectangulo["x1"][i]),(rectangulo["y2"][i],rectangulo["x1"][i]),(rectangulo["y2"][i],rectangulo["x2"][i]),(rectangulo["y1"][i],rectangulo["x2"][i])]))
gdf1=gpd.GeoDataFrame(geometry=geometry,crs=toluca.crs)
gdf1.set_crs=toluca.crs
gdf1.to_file("C:/Users/ASUS/Desktop/zerodayssoftware/deteccion de terrenos/poligonos.shp")

In [6]:
# import sys
# sys.path.append("C:/Users/ASUS/Inteligencia_Artificial/yolov7")
import os
os.chdir("C:/Users/ASUS/Inteligencia_Artificial/yolov7")
import yolo_Alberto
yolo_Alberto.detect(weights="C:/Users/ASUS/Inteligencia_Artificial/yolov7/best25.pt")



  0%|          | 0/20 [00:00<?, ?it/s]

C:\Users\ASUS\Inteligencia_Artificial\yolov7\inference\images\bus.jpg


  5%|▌         | 1/20 [00:00<00:05,  3.52it/s]

5
C:\Users\ASUS\Inteligencia_Artificial\yolov7\inference\images\horses.jpg


 10%|█         | 2/20 [00:00<00:04,  3.97it/s]

...

C:\Users\ASUS\Inteligencia_Artificial\yolov7\inference\images\zona_4_0_9.png


 95%|█████████▌| 19/20 [00:07<00:00,  2.04it/s]

C:\Users\ASUS\Inteligencia_Artificial\yolov7\inference\images\zona_4_6_28.png


100%|██████████| 20/20 [00:07<00:00,  2.68it/s]


[[tensor(0.),
  0.04753086343407631,
  0.6592592597007751,
  0.09506172686815262,
  0.2925925850868225,
  'bus'],
 [tensor(0.),
  0.9117283821105957,
  0.5874999761581421,
  0.1765432059764862,
  0.4472222328186035,
  'bus'],
 [tensor(0.),
  0.3493827283382416,
  0.5874999761581421,
  0.15061728656291962,
  0.42129629850387573,
  'bus'],
 [tensor(0.),
  0.17901234328746796,
  0.6037036776542664,
  0.2419753074645996,
  0.4740740656852722,
  'bus'],
 [tensor(5.),
  0.5055555701255798,
  0.4444444477558136,
  0.9839506149291992,
  0.46481481194496155,
  'bus'],
 [tensor(17.),
  0.20504528284072876,
  0.5869140625,
  0.40750324726104736,
  0.423828125,
  'horses'],
 [tensor(0.),
  0.3148437440395355,
  0.5009328126907349,
  0.6265624761581421,
  0.986940324306488,
  'image1'],
 [tensor(0.),
  0.690625011920929,
  0.5718283653259277,
  0.609375,
  0.8488805890083313,
  'image1'],
 [tensor(0.),
  0.37968748807907104,
  0.5166666507720947,
  0.6781250238418579,
  0.9583333134651184,
  'image

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


100%|██████████| 20/20 [00:07<00:00,  2.77it/s]


[[tensor(0.),
  0.14074073731899261,
  0.520370364189148,
  0.15802469849586487,
  0.2648148238658905,
  'bus'],
 [tensor(0.),
  0.054320987313985825,
  0.7662037014961243,
  0.0962962955236435,
  0.13611111044883728,
  'bus'],
 [tensor(0.),
  0.1537037044763565,
  0.41342592239379883,
  0.10493826866149902,
  0.06018518656492233,
  'bus'],
 [tensor(0.),
  0.8216049671173096,
  0.15694443881511688,
  0.287654310464859,
  0.15833333134651184,
  'bus'],
 [tensor(0.),
  0.15493826568126678,
  0.7662037014961243,
  0.13209876418113708,
  0.13425925374031067,
  'bus'],
 [tensor(0.),
  0.27407407760620117,
  0.7425925731658936,
  0.1061728373169899,
  0.09814814478158951,
  'bus'],
 [tensor(0.),
  0.8808642029762268,
  0.7458333373069763,
  0.23827160894870758,
  0.5083333253860474,
  'bus'],
 [tensor(0.),
  0.3580246865749359,
  0.1111111119389534,
  0.18518517911434174,
  0.21111111342906952,
  'bus'],
 [tensor(0.),
  0.5117284059524536,
  0.6796296238899231,
  0.9567901492118835,
  0.3462

100%|██████████| 20/20 [00:06<00:00,  2.93it/s]


[[tensor(0.),
  0.14074073731899261,
  0.520370364189148,
  0.15802469849586487,
  0.2648148238658905,
  'bus'],
 [tensor(0.),
  0.054320987313985825,
  0.7662037014961243,
  0.0962962955236435,
  0.13611111044883728,
  'bus'],
 [tensor(0.),
  0.1537037044763565,
  0.41342592239379883,
  0.10493826866149902,
  0.06018518656492233,
  'bus'],
 [tensor(0.),
  0.8216049671173096,
  0.15694443881511688,
  0.287654310464859,
  0.15833333134651184,
  'bus'],
 [tensor(0.),
  0.15493826568126678,
  0.7662037014961243,
  0.13209876418113708,
  0.13425925374031067,
  'bus'],
 [tensor(0.),
  0.27407407760620117,
  0.7425925731658936,
  0.1061728373169899,
  0.09814814478158951,
  'bus'],
 [tensor(0.),
  0.8808642029762268,
  0.7458333373069763,
  0.23827160894870758,
  0.5083333253860474,
  'bus'],
 [tensor(0.),
  0.3580246865749359,
  0.1111111119389534,
  0.18518517911434174,
  0.21111111342906952,
  'bus'],
 [tensor(0.),
  0.5117284059524536,
  0.6796296238899231,
  0.9567901492118835,
  0.3462

In [7]:
vector=0
%run -i detect.py --weights best25.pt --conf 0.1 --img-size 256 --source inference/images/


YOLOR  v0.1-121-g2fdc7f1 torch 1.12.1+cpu CPU



Namespace(weights=['best25.pt'], source='inference/images/', img_size=256, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


Model Summary: 314 layers, 36487166 parameters, 6194944 gradients, 103.2 GFLOPS


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

13 casas, Done. (369.6ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp43\bus.jpg
2 terrenos, Done. (335.2ms) Inference, (5.5ms) NMS
 The image with the result is saved in: runs\detect\exp43\horses.jpg
3 casas, Done. (347.6ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp43\image1.jpg
3 casas, 2 terrenos, Done. (297.0ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp43\image2.jpg
2 casas, 5 terrenos, Done. (198.1ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp43\image3.jpg
4 casas, Done. (224.5ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp43\zidane.jpg
2 terrenos, Done. (314.6ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp43\zona_4_0_10.png
1 terreno, Done. (404.4ms) Inference, (3.5ms) NMS
 The image wi

In [ ]:
(weights=['best25.pt'], source='inference/images/', img_size=256, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)

In [206]:
s=[]
for v in vector:
    str_v=(str(v).replace("tensor(","").replace("[","").replace(".)","").replace(")","").replace("(","").replace("']","").replace("'","").strip().split(","))
    h,w=1024,1024
    x1 = int( float(str_v[1]) * w )
    y1 = int( float(str_v[2]) * h )
    xw = int( float(str_v[3]) * w /2)
    yw = int( float(str_v[4]) * h /2)
    start_point = ((x1 - xw)/w, (y1 - yw)/h )
    end_point   = ((x1 + xw)/w, (y1 + yw)/h )
    if str(str_v[0])=="0":
        tipo="casa"
    else:
        tipo="terreno"
    # print(tipo,start_point,end_point)
    s.append([tipo,start_point,end_point,str_v[5].strip()])
resultado_nogeo=pd.DataFrame(s,columns=["Tipo","start_point","end_point","imagen"])
resultado_nogeo

,Tipo,start_point,end_point,imagen
0,casa,"(0.185546875, 0.7470703125)","(0.349609375, 0.8154296875)",centro_0_0
1,casa,"(0.0078125, 0.4033203125)","(0.125, 0.5087890625)",centro_0_0
2,casa,"(0.1845703125, 0.5595703125)","(0.2685546875, 0.6220703125)",centro_0_0
3,casa,"(0.62890625, 0.00390625)","(0.69921875, 0.09765625)",centro_0_0
4,terreno,"(0.4033203125, 0.3271484375)","(0.5654296875, 0.5322265625)",centro_0_0
...,...,...,...,...
23901,casa,"(0.0712890625, 0.630859375)","(0.3466796875, 0.7265625)",zona_6_alfredo_9_9
23902,casa,"(0.7216796875, 0.73828125)","(0.8505859375, 0.82421875)",zona_6_alfredo_9_9
23903,casa,"(0.5869140625, 0.392578125)","(0.8603515625, 0.51953125)",zona_6_alfredo_9_9
23904,casa,"(0.7177734375, 0.83203125)","(0.8466796875, 0.912109375)",zona_6_alfredo_9_9


In [207]:
import geopandas as gpd
resultdo_geo=gpd.read_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7\runs\detect/centro.shp")
resultdo_geo.columns=["imagen","geometry"]
resultdo_geo

,imagen,geometry
0,centro_0_0,"POLYGON ((-11095642.832 2189921.416, -11095537..."
1,centro_0_1,"POLYGON ((-11095537.815 2189921.416, -11095432..."
2,centro_0_2,"POLYGON ((-11095432.798 2189921.416, -11095327..."
3,centro_0_3,"POLYGON ((-11095327.781 2189921.416, -11095222..."
4,centro_0_4,"POLYGON ((-11095222.764 2189921.416, -11095117..."
...,...,...
607,centro_17_29,"POLYGON ((-11092597.338 2188119.489, -11092492..."
608,centro_17_30,"POLYGON ((-11092492.321 2188119.489, -11092387..."
609,centro_17_31,"POLYGON ((-11092387.304 2188119.489, -11092282..."
610,centro_17_32,"POLYGON ((-11092282.286 2188119.489, -11092177..."


In [212]:
import json
from shapely.geometry import Polygon, mapping
cruce=pd.merge(resultado_nogeo,resultdo_geo)
# cruce=cruce[cruce["imagen"]=="centro_0_0"]
len(cruce)



13043

In [215]:
tipos=["casa","terreno"]
import tqdm
geometry=[]
for tipo in tipos:
    df_aux=cruce[cruce["Tipo"]==tipo].reset_index(drop=True)
    for i in tqdm.tqdm(range(len(df_aux))):
        coordenadas=mapping(df_aux["geometry"][i])["coordinates"][0]
        coordenadas=np.array(coordenadas)
        maxy=np.max(coordenadas[:,0])
        miny=np.min(coordenadas[:,0])
        maxx=np.max(coordenadas[:,1])
        minx=np.min(coordenadas[:,1])
        coordenadas=[(cruce["start_point"][i][1],cruce["start_point"][i][0],cruce["end_point"][i][1],cruce["end_point"][i][0])]
        rectangulo=pd.DataFrame(coordenadas,columns=["x1","y1","x2","y2"])
        rectangulo["x1"]=maxx-rectangulo["x1"]*(maxx-minx)
        rectangulo["y1"]=miny+rectangulo["y1"]*(maxy-miny)
        rectangulo["x2"]=maxx-rectangulo["x2"]*(maxx-minx)
        rectangulo["y2"]=miny+rectangulo["y2"]*(maxy-miny)
        geometry.append(Polygon([(rectangulo["y1"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x2"][0]),(rectangulo["y1"][0],rectangulo["x2"][0])]))
        # startx,starty=map(cruce["start_point"][i][0],0,1024,minx,maxx),map(cruce["start_point"][i][1],1024,0,miny,maxy)
        # endx,endy=map(cruce["end_point"][i][0],0,1024,minx,maxx),map(cruce["end_point"][i][1],1024,0,miny,maxy)
        # geometry.append(Polygon([(starty,startx),(endy,startx),(endy,endx),(starty,endx),(starty,startx)]))
    gdf1=gpd.GeoDataFrame(geometry=geometry,crs=resultdo_geo.crs)
    gdf1.set_crs=resultdo_geo.crs
    gdf1.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7\runs\detect/poligonos_"+tipo+".shp")

100%|██████████| 414/414 [00:00<00:00, 490.83it/s]


In [193]:
coordenadas=list(coordenadas[0])
[int(x)/1024 for x in coordenadas]

[0.3837890625, 0.2158203125, 0.4736328125, 0.3857421875]

In [171]:
    coordenadas

array([[-1.1096e+07,  2.1899e+06],
       [-1.1096e+07,  2.1899e+06],
       [-1.1096e+07,  2.1898e+06],
       [-1.1096e+07,  2.1898e+06],
       [-1.1096e+07,  2.1899e+06]])

0    LINESTRING (-11095575.761 2189854.961, -110955...
1    LINESTRING (-11095586.221 2189816.352, -110955...
2    LINESTRING (-11095574.940 2189910.340, -110955...
3    LINESTRING (-11095563.351 2189885.497, -110955...
4    LINESTRING (-11095556.993 2189834.259, -110955...
5    LINESTRING (-11095556.685 2189898.333, -110955...
6    LINESTRING (-11095596.682 2189894.813, -110955...
7    LINESTRING (-11095621.808 2189897.401, -110956...
8    LINESTRING (-11095631.346 2189819.560, -110956...
9    LINESTRING (-11095551.865 2189854.444, -110955...
dtype: geometry

In [21]:
import glob
files=glob.glob(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7\train\labelss2/*.txt")
# files=r"""C:\Users\ASUS\Inteligencia_Artificial\yolov7\train\labels\centro_16_1.txt
# C:\Users\ASUS\Inteligencia_Artificial\yolov7\train\labels\centro_16_3.txt
# C:\Users\ASUS\Inteligencia_Artificial\yolov7\train\labels\centro_16_3.txt""".split("\n")
files
for i in range(len(files)):
    f=open(files[i])
    lines=f.readlines()
    f.close()
    new=[]
    for line in lines:
        line=line.replace("\n","")
        if  str(line[0])=="0":
            pass
        elif str(line[0])=="1" or str(line[0])=="3":# str(line[0])=="0" or 
            line="0 "+" ".join(line.split()[1:])
            print(files[i])
        else:
            line="1 "+" ".join(line.split()[1:])
        new.append(line)
    f=open("C:/Users/ASUS/Inteligencia_Artificial/yolov7/train/labels/"+files[i].replace("\\","/").split("/")[-1],"w")
    f.write("\n".join(new))
    f.close()



In [ ]:
import cv2
import glob
import imutils
path="C:/Users/ASUS/Inteligencia_Artificial/yolov7/train/"
images=glob.glob(path+"images/*")
labels=glob.glob(path+"labels/*")
la=[str(x).replace("\\","/").split("/")[-1][:-4] for x in labels]
for i in range(len(la)):
    count=0
    try:
        img = cv2.imread(path+"images/"+la[i]+".png")
        h,w,_ = img.shape
    except:
        continue
    with open(path+"labels/"+la[i]+".txt") as f:
        lines=f.readlines()
    for line in lines:    
        
        x1 = int( float(line.split()[1]) * w )
        y1 = int( float(line.split()[2]) * h )
        xw = int( float(line.split()[3]) * w /2)
        yw = int( float(line.split()[4]) * h /2)
        start_point = (x1 - xw, y1 - yw )
        end_point   = (x1 + xw, y1 + yw )
        # cv2.rectangle(img, start_point, end_point, (255, 0, 0), 2)
        imAux = img.copy()
        objeto = imAux[start_point[1]:end_point[1],start_point[0]:end_point[0]]
        objeto = imutils.resize(objeto,width=38)
        if str(line.split()[0])=="0":
            cv2.imwrite(path+"haar/p/"+str(i)+"_"+str(count)+".png",objeto)
        else:
            cv2.imwrite(path+"haar/n/"+str(i)+"_"+str(count)+".png",objeto)
        count+=1
    # cv2.imshow('frame',img)
    # cv2.waitKey()
    # cv2.destroyAllWindows()


In [ ]:
la[25]

In [ ]:
line.split()[0]

In [1]:
import os
os.chdir(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7")
import argparse
import time
from pathlib import Path
import tqdm
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

# def detect(source, weights):
source=r"C:\Users\ASUS\Inteligencia_Artificial\yolov7\valid\images"
weights="best.pt"
device="cpu"
trace=True
source, weights=source, weights
dataset = LoadImages(source, img_size=256, stride=1)
model = attempt_load(weights, map_location=device)
names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
for path, img, im0s, vid_cap in tqdm.tqdm(dataset):
    img = torch.from_numpy(img).to("cpu")
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
        pred = model(img, augment=True)[0]
        
    for i, det in enumerate(pred):  # detections per image
        p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)
        p = Path(p)  # to Path
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

            # Write results
            for *xyxy, conf, cls in reversed(det):
                xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format
                vector.append(list(line)+[p.name[:-4]])

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


  0%|          | 0/5 [00:01<?, ?it/s]


RuntimeError: shape '[1, 4]' is invalid for input of size 5

In [12]:
webcam=False
if webcam:
    s=1
else:
    s=0
s

0